# MESQUAL 201: KPI Framework and Unit Handling

This notebook demonstrates how to build a structured KPI (Key Performance Indicator) system with proper unit handling using MESQUAL's new KPI framework.

## Introduction

MESQUAL's KPI framework provides a sophisticated system for creating, managing, and analyzing performance metrics across energy scenarios. This notebook covers:

1. **KPI Builder Pattern**: Declarative KPI creation with fluent API
2. **Batch Generation**: Efficient computation across multiple objects
3. **Unit Handling**: Automatic unit conversion and pretty formatting
4. **Aggregations**: Various statistical aggregations (Sum, Mean, Max, etc.)
5. **KPI Attributes**: Rich metadata for filtering and organization
6. **Comparison KPIs**: Scenario comparison workflows

The new KPI system is designed for **10x performance improvement** through batch-first architecture while providing a cleaner, more intuitive API.

## Setup

First, we need to set up the environment. If you are on Colab, the first cell will clone and install all dependencies. You will have to restart the session afterwards and continue with cell 2. If you are in a local environment, make sure that you have followed the Getting started steps in the README, so that mesqual and all requirements are installed.

In [ ]:
import os

if "COLAB_RELEASE_TAG" in os.environ:
    import importlib.util

    def is_module_available(module_name):
        return importlib.util.find_spec(module_name) is not None

    if os.path.exists("mesqual-vanilla-studies") and is_module_available("mesqual"):
        print("✅ Environment already set up. Skipping installation.")
    else:
        print("🔧 Setting up Colab environment...")
        !git clone --recursive https://github.com/helgeesch/mesqual-vanilla-studies.git
        %cd mesqual-vanilla-studies/

        !pip install git+https://github.com/helgeesch/mesqual -U
        !pip install git+https://github.com/helgeesch/mesqual-pypsa -U
        !pip install git+https://github.com/helgeesch/captain-arro -U
        !pip install -r requirements.txt

        print('✅ Setup complete. 🔁 Restart the session, then skip this cell and continue with the next one.')
else:
    print("🖥️ Running locally. No setup needed.")

In [ ]:
import os

if "COLAB_RELEASE_TAG" in os.environ:
    import sys
    sys.path.append('/content/mesqual-vanilla-studies')
    os.chdir('/content/mesqual-vanilla-studies')
else:
    def setup_notebook_env():
        """Set working directory to repo root and ensure it's in sys.path."""
        import os
        import sys
        from pathlib import Path

        def find_repo_root(start_path: Path) -> Path:
            current = start_path.resolve()
            while current != current.parent:
                if (current / 'vanilla').exists():
                    return current
                current = current.parent
            raise FileNotFoundError(f"Repository root not found from: {start_path}")

        repo_root = find_repo_root(Path.cwd())
        os.chdir(repo_root)
        if str(repo_root) not in sys.path:
            sys.path.insert(0, str(repo_root))

    setup_notebook_env()

try:
    from mesqual import StudyManager
except ImportError:
    raise ImportError("❌ 'mesqual' not found. If you're running locally, make sure you've installed all dependencies as described in the README.")

if not os.path.isdir("studies"):
    raise RuntimeError(f"❌ 'studies' folder not found. Make sure your working directory is set to the mesqual-vanilla-studies root. Current working directory: {os.getcwd()}")

print("✅ Environment ready. Let's go!")

In [ ]:
import pandas as pd
from mesqual.kpis import FlagAggKPIBuilder, Aggregations
from mesqual.units import Units
from vanilla.notebook_config import configure_clean_output_for_jupyter_notebook

configure_clean_output_for_jupyter_notebook()

## Load Study Data

Let's load our study data using the same Scigrid-DE setup:

In [ ]:
from studies.study_01_intro_to_mesqual.scripts.setup_study_manager import get_scigrid_de_study_manager

study = get_scigrid_de_study_manager()

print("Study scenarios:")
for dataset in study.scen.datasets:
    print(f"  📊 {dataset.name}")

# Get base dataset for exploration
ds_base = study.scen.get_dataset('base')
print(f"\n✅ Loaded dataset: {ds_base.name}")

## Part 1: Building KPIs with the Builder Pattern

The KPI framework uses a fluent builder pattern for declarative KPI creation:

### Creating a Simple KPI

Let's create a KPI for average market price in a control area:

In [ ]:
# Define KPI using builder pattern
price_kpi_definition = (
    FlagAggKPIBuilder()
    .for_flag('control_areas_t.vol_weighted_marginal_price')
    .with_aggregation(Aggregations.Mean)
    .for_object('TenneTDE')
    .build()
)

# Generate KPI
kpis = price_kpi_definition.generate_kpis(ds_base)

# Examine the KPI
kpi = kpis[0]
print(f"KPI Name: {kpi.name}")
print(f"Value: {kpi.value:.2f}")
print(f"Unit: {kpi.unit}")
print(f"Quantity: {kpi.quantity}")
print(f"\nKPI Attributes:")
for attr, value in kpi.attributes.as_dict(primitive_values=True).items():
    print(f"  {attr}: {value}")

### Batch KPI Generation

The real power comes from batch generation across multiple objects:

In [ ]:
# Get all control areas
control_areas = ds_base.fetch('control_areas').index.to_list()
print(f"Control areas: {control_areas}")

# Create KPIs for all control areas at once
all_price_kpis_def = (
    FlagAggKPIBuilder()
    .for_flag('control_areas_t.vol_weighted_marginal_price')
    .with_aggregation(Aggregations.Mean)
    .for_objects(control_areas)  # Batch generation
    .build()
)

all_price_kpis = all_price_kpis_def.generate_kpis(ds_base)

print(f"\n✅ Generated {all_price_kpis.size} KPIs in one operation")
print("\nSample KPIs:")
for kpi in all_price_kpis[:3]:
    print(f"  {kpi.name}: {kpi.value:.2f} {kpi.unit}")

## Part 2: Aggregation Types

MESQUAL supports various aggregation types:

In [ ]:
# Create KPIs with different aggregations
aggregations = [Aggregations.Sum, Aggregations.Mean, Aggregations.Max, Aggregations.Min]

for agg in aggregations:
    kpi_def = (
        FlagAggKPIBuilder()
        .for_flag('control_areas_t.vol_weighted_marginal_price')
        .with_aggregation(agg)
        .for_object('TenneTDE')
        .build()
    )
    kpi = kpi_def.generate_kpis(ds_base)[0]
    print(f"{agg.name:10s}: {kpi.value:8.2f} {kpi.unit}")

## Part 3: Unit Handling

MESQUAL's unit system provides automatic conversion and pretty formatting:

### Automatic Pretty Units

In [ ]:
# Create generation KPI
gen_kpi_def = (
    FlagAggKPIBuilder()
    .for_flag('generators_t.p')
    .with_aggregation(Aggregations.Sum)
    .for_all_objects()
    .build()
)

gen_kpis = gen_kpi_def.generate_kpis(ds_base)

# Show original and pretty units
sample_kpi = gen_kpis[0]
print(f"Original quantity: {sample_kpi.quantity}")
print(f"Pretty unit: {Units.get_quantity_in_pretty_unit(sample_kpi.quantity)}")
print(f"Pretty text: {Units.get_pretty_text_for_quantity(Units.get_quantity_in_pretty_unit(sample_kpi.quantity), thousands_separator=' ')}")

### KPI Unit Conversion Methods

In [ ]:
# Get a KPI with energy units
sample_kpi = gen_kpis[0]

print("Original KPI:")
print(f"  Value: {sample_kpi.value:.2f}")
print(f"  Unit: {sample_kpi.unit}")
print(f"  Quantity: {sample_kpi.quantity}")

# Convert to different unit
kpi_in_gwh = sample_kpi.get_kpi_in_unit(Units.GWh)
print(f"\nConverted to GWh:")
print(f"  Value: {kpi_in_gwh.value:.2f}")
print(f"  Unit: {kpi_in_gwh.unit}")

# Get in pretty unit
kpi_pretty = sample_kpi.get_kpi_in_pretty_unit()
print(f"\nPretty unit:")
print(f"  Value: {kpi_pretty.value:.2f}")
print(f"  Unit: {kpi_pretty.unit}")

## Part 4: KPI Attributes and Metadata

Every KPI carries rich metadata for filtering and organization:

In [ ]:
# Create KPI with custom attributes
kpi_def = (
    FlagAggKPIBuilder()
    .for_flag('control_areas_t.vol_weighted_marginal_price')
    .with_aggregation(Aggregations.Mean)
    .for_object('TenneTDE')
    .with_attributes(
        name_prefix='Regional',
        custom_category='Market Analysis'
    )
    .build()
)

kpi = kpi_def.generate_kpis(ds_base)[0]

print("KPI Attributes:")
attrs = kpi.attributes.as_dict(primitive_values=True)
for key, value in sorted(attrs.items()):
    if value is not None:
        print(f"  {key:20s}: {value}")

## Part 5: Multi-Scenario KPI Generation

Generate KPIs across multiple scenarios efficiently:

In [ ]:
# Clear existing KPIs
study.scen.clear_kpi_collection_for_all_child_datasets()

# Generate price KPIs for all scenarios
price_def = (
    FlagAggKPIBuilder()
    .for_flag('control_areas_t.vol_weighted_marginal_price')
    .with_aggregation(Aggregations.Mean)
    .for_all_objects()
    .build()
)

for dataset in study.scen.datasets:
    kpis = price_def.generate_kpis(dataset)
    dataset.kpi_collection.extend(kpis)
    print(f"✅ Generated {kpis.size} KPIs for {dataset.name}")

# Get merged collection
all_kpis = study.scen.get_merged_kpi_collection()
print(f"\n📊 Total KPIs across all scenarios: {all_kpis.size}")

## Part 6: KPI Collection Filtering

The KPI collection provides powerful filtering capabilities:

In [ ]:
# Filter by attributes
tennet_kpis = all_kpis.filter(object_name='TenneTDE')
print(f"KPIs for TenneTDE: {tennet_kpis.size}")

# Filter by dataset
base_kpis = all_kpis.filter(dataset_name='base')
print(f"KPIs for base scenario: {base_kpis.size}")

# Combined filtering
tennet_base = all_kpis.filter(
    object_name='TenneTDE',
    dataset_name='base',
    aggregation=Aggregations.Mean
)
print(f"\nFiltered result: {tennet_base.size} KPI(s)")
if tennet_base.size > 0:
    kpi = tennet_base[0]
    print(f"  {kpi.name}: {kpi.value:.2f} {kpi.unit}")

## Part 7: KPI Grouping

Group KPIs by attributes for analysis:

In [ ]:
# Group by dataset
by_dataset = all_kpis.group_by('dataset_name')

print("KPIs grouped by dataset:")
for (dataset,), kpi_collection in by_dataset.items():
    print(f"  {dataset}: {kpi_collection.size} KPIs")

# Group by object and dataset
by_object_dataset = all_kpis.group_by('object_name', 'dataset_name')

print(f"\nTotal groups (object × dataset): {len(by_object_dataset)}")
print("\nSample groups:")
for i, ((obj, ds), kpis) in enumerate(by_object_dataset.items()):
    if i < 3:
        print(f"  {obj} × {ds}: {kpis.size} KPI(s)")

## Summary: KPI Framework Capabilities

### Key Features Demonstrated:

1. **Fluent Builder API**
   - `.for_flag()` - Specify data source
   - `.with_aggregation()` - Choose aggregation type
   - `.for_object()` / `.for_objects()` - Select objects
   - `.with_attributes()` - Add custom metadata
   - `.build()` - Create definition

2. **Batch Operations**
   - Single fetch per flag (not per object)
   - Column-wise aggregation
   - 10x performance improvement

3. **Unit Handling**
   - Automatic unit tracking
   - Pretty unit conversion
   - `get_kpi_in_unit()` - Convert to specific unit
   - `get_kpi_in_pretty_unit()` - Auto-select readable unit

4. **Rich Metadata**
   - Object name, flag, aggregation
   - Dataset name and type
   - Custom attributes
   - Extra attributes dictionary

5. **Collection Operations**
   - `.filter()` - Filter by attributes
   - `.group_by()` - Group by attributes
   - `.get_related()` - Find related KPIs
   - `.to_dataframe()` - Export to pandas

### Performance Benefits:

- **Old System**: N fetches + N aggregations (one per object)
- **New System**: 1 fetch + 1 aggregation (batch operation)
- **Result**: Up to 10x faster for large datasets

### Next Steps:

In notebook 202, we'll explore:
- Exporting KPI collections to DataFrames
- Creating pretty tables with automatic unit normalization
- Comparison KPIs for scenario analysis
- Advanced filtering and visualization

## Conclusion

The MESQUAL KPI framework provides:

✅ **Clean, declarative API** - Builder pattern for readable code
✅ **High performance** - Batch operations for efficiency
✅ **Automatic unit handling** - No manual conversion needed
✅ **Rich metadata** - Everything you need for filtering and analysis
✅ **Flexible filtering** - Find exactly the KPIs you need

This foundation enables sophisticated multi-scenario energy system analysis with minimal code.